# Previsão de resultados do Brasileirão

Este código busca utilizar modelos de aprendizado de máquina para predizer o resultado de jogos do Brasileirão.

In [678]:
# importanto algumas bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [679]:
# importanto o dataframe

# Código para ler o arquivo em xls e retorná-lo em dataframe
dfbr = pd.read_excel("dados brasileirao.xlsx") 

c:\venv\ilumpy\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [680]:
nan = dfbr.loc[0]["arbitro"]

In [681]:
# Nesta linha, foram tiradas todas as linhas que possuiam dados faltantes, visto que os anos iniciais não tinham todos os dados da tabela preenchidos.
dfbr = dfbr.dropna(axis = 0)

### Minha ideia

Eu vou dividir o dataframe entre os times. Então, para cada jogo, eu associarei a tendência de gols, escanteios, chutes, escanteios, entre outros, de cada time. Dessa forma, após a adição das tendências, eu terei um dataframe parecido com o inicial, porém, com uma adição de diversas novas features que tornarão o modelo de previsão mais completo.

### Execução

In [682]:
def tendencia_de_uma_lista(lista_feature, razao):
    divisor = 1*(1 - razao**4)/(1 - razao)
    soma = 0
    for i in range(4):
        termo = lista_feature[-(i+1)]*(razao**(i))
        soma = soma + termo
        
    tendencia = soma/divisor
    
    return round(tendencia, 3)

In [683]:
def feature_conjugada(feature):
    if feature == 'gols_man':
        return 'gols_vis', 'time_man'
    
    elif feature == 'gols_vis':
        return 'gols_man', 'time_vis'
    
    elif feature == 'gols_1_tempo_man':
        return 'gols_1_tempo_vis','time_man'
    
    elif feature == 'gols_1_tempo_vis':
        return 'gols_1_tempo_man', 'time_vis'
    
    elif feature == 'escanteios_man':
        return 'escanteios_vis', 'time_man'
    
    elif feature == 'escanteios_vis':
        return 'escanteios_man', 'time_vis'
    
    elif feature == 'faltas_man':
        return 'faltas_vis', 'time_man'
    
    elif feature == 'faltas_vis':
        return 'faltas_man', 'time_vis'
        
    elif feature == 'chutes_bola_parada_man':
        return 'chutes_bola_parada_vis', 'time_man'
        
    elif feature == 'chutes_bola_parada_vis':
        return 'chutes_bola_parada_man', 'time_vis'
    
    elif feature == 'defesas_man':
        return 'defesas_vis', 'time_man'
    
    elif feature == 'defesas_vis':
        return 'defesas_man', 'time_vis'
    
    elif feature == 'impedimentos_man':
        return 'impedimentos_vis', 'time_man'
    
    elif feature == 'impedimentos_vis':
        return 'impedimentos_man', 'time_vis'
    
    elif feature == 'chutes_man':
        return 'chutes_vis', 'time_man'
    
    elif feature == 'chutes_vis':
        return 'chutes_man', 'time_vis'
        
    elif feature == 'chutes_fora_man':
        return 'chutes_fora_vis', 'time_man'
    
    elif feature == 'chutes_fora_vis':
        return 'chutes_fora_man', 'time_vis'
        
    else:
        print("Você tem certeza que essa feature faz parte do dataframe e que possui conjugado?")
        return False, False

In [684]:
lista_das_features = ['gols_man', 'gols_1_tempo_man', 'escanteios_man', 'faltas_man', 'chutes_bola_parada_man', 'defesas_man', 'impedimentos_man', 'chutes_man', 'chutes_fora_man']

In [685]:
novas_colunas = []

for feat in lista_das_features:
    feat_conj = feature_conjugada(feat)[0]
    nome_nova_coluna = "tendencia_" + feat
    nome_nova_coluna_conj = "tendencia_" + feat_conj
    novas_colunas.append(nome_nova_coluna)
    novas_colunas.append(nome_nova_coluna_conj)

In [686]:
for coluna in novas_colunas:
    dfbr[coluna] = nan

In [687]:
def tendencia(index, razao):
    
    # Análise do time mandante
    time_man = dfbr.loc[index]["time_man"] # time mandante
    #print(time_man)
    df_anteriores_man = dfbr.loc[((dfbr["time_man"] == time_man) | (dfbr["time_vis"] == time_man)) & (dfbr.index < index)]
    
    # Análise do time visitante
    time_vis = dfbr.loc[index]["time_vis"] # time visitante
    
    #print(time_vis)
    
    df_anteriores_vis = dfbr.loc[((dfbr["time_man"] == time_vis) | (dfbr["time_vis"] == time_vis)) & (dfbr.index < index)]
    
    #print(df_anteriores_vis)
    
    for feature in lista_das_features:
        
        #print(feature)
        
        feature_conj = feature_conjugada(feature)[0]
        
        lista_feature_time = []
        
        for i in df_anteriores_man.index:
    
            if df_anteriores_man["time_man"][i] == time_man:
                lista_feature_time.append(df_anteriores_man[feature][i])
        
            if df_anteriores_man["time_vis"][i] == time_man:
                lista_feature_time.append(df_anteriores_man[feature_conj][i])
                
        lista_feature_conj_time = []
        
        for j in df_anteriores_vis.index:
            
            #print(df_anteriores_vis["time_vis"][j])

            if df_anteriores_vis["time_man"][j] == time_vis:
                lista_feature_conj_time.append(df_anteriores_vis[feature][j])
        
            if df_anteriores_vis["time_vis"][j] == time_vis:
                lista_feature_conj_time.append(df_anteriores_vis[feature_conj][j])
                
        if len(lista_feature_time) >= 4 and len(lista_feature_conj_time) >= 4:
            
            #print(lista_feature_time)
            
            tendencia_feature = tendencia_de_uma_lista(lista_feature_time, razao)
                    
            #print(tendencia_feature)
            
            #print(lista_feature_conj_time)
            
            tendencia_feature_conj = tendencia_de_uma_lista(lista_feature_conj_time, razao)
            
            nome_coluna = "tendencia_" + feature
            
            nome_coluna_conj = "tendencia_" + feature_conj
            
            dfbr.at[index, nome_coluna] = tendencia_feature
            
            dfbr.at[index, nome_coluna_conj] = tendencia_feature_conj
            
    return 

In [688]:
for i in dfbr.index:
    tendencia(i, 0.5)

In [689]:
# Nesta linha, novamente, foram tiradas todas as linhas que possuiam dados faltantes, visto que os anos iniciais não tinham todos os dados da tabela preenchidos.
dfbr = dfbr.dropna(axis = 0)

In [690]:
# Nesta linha, foram retiradas todas as colunas que possuiam dados relativos à estáticas após o começo do jogo. Os Dados após o jogo começar não nos ajudam a prever os nossos resultados, pois são muito específicos de cada jogo. Então vamos analisar o principal: número de gols
dfbr = dfbr.drop(["escanteios_man", "escanteios_vis", "publico_max", "faltas_man", "faltas_vis", "chutes_bola_parada_man", "chutes_bola_parada_vis", "defesas_man", "defesas_vis", "impedimentos_man", "impedimentos_vis", "chutes_man", "chutes_vis", "chutes_fora_man", "chutes_fora_vis", "gols_1_tempo_man", "gols_1_tempo_vis", 'Unnamed: 0', 'ano_campeonato', 'data', 'horario', 'estadio', 'arbitro', 'time_man', 'time_vis', 'tecnico_man', 'tecnico_vis', 'rodada'], axis=1)

### Machine Learning

In [691]:
# Divisão dos dados do dataset em treino e teste

from sklearn.model_selection import train_test_split # Divisão feita usando a função train_test_split

TAMANHO_TESTE = 0.1 # Porcentagem destinada do dataset ao grupo de teste
SEMENTE_ALEATORIA = 122106 # Definição de uma seed para evitar mudança desses grupos

# Escolha dos índices e seleção feita pela função
indices = dfbr.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Atribuição dos índices escolhidos aos dados
dfbr_treino = dfbr.loc[indices_treino].sort_index()
dfbr_teste = dfbr.loc[indices_teste].sort_index()

# Aqui são escolhidos os atributos do dataset e é definido o target 

FEATURES = ['publico', 'colocacao_man', 'colocacao_vis', 'valor_equipe_titular_man',
       'valor_equipe_titular_vis', 'idade_media_titular_man',
       'idade_media_titular_vis', 'tendencia_gols_man',
       'tendencia_gols_vis', 'tendencia_gols_1_tempo_man',
       'tendencia_gols_1_tempo_vis', 'tendencia_escanteios_man',
       'tendencia_escanteios_vis', 'tendencia_faltas_man',
       'tendencia_faltas_vis', 'tendencia_chutes_bola_parada_man',
       'tendencia_chutes_bola_parada_vis', 'tendencia_defesas_man',
       'tendencia_defesas_vis', 'tendencia_impedimentos_man',
       'tendencia_impedimentos_vis', 'tendencia_chutes_man',
       'tendencia_chutes_vis', 'tendencia_chutes_fora_man',
       'tendencia_chutes_fora_vis']
TARGET = ["gols_man"]

# Divide os dados para se adequarem aos modelos

X_treino = dfbr_treino.reindex(FEATURES, axis=1).values
y_treino = dfbr_treino.reindex(TARGET, axis=1).values
X_teste = dfbr_teste.reindex(FEATURES, axis=1).values
y_teste = dfbr_teste.reindex(TARGET, axis=1).values

### Modelo Baseline

In [692]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

# Cria um modelo baseline para termos uma base de comparação com outros modelos
modelo_baseline = DummyRegressor()

# Treina o modelo de acordo com os dados escolhidos antes
modelo_baseline.fit(X_treino, y_treino)

# Realiza uma previsão usando o modelo treinado
previsao_baseline = modelo_baseline.predict(X_teste)
print(f"O time mandante faria {previsao_baseline[0]} gols")

# Verificando o erro do modelo usando o RMSE (root mean squared error)
y_verdadeiro = y_teste
y_previsao_baseline = modelo_baseline.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao_baseline, squared=False)

print(f"O RMSE do modelo baseline foi de {RMSE} gols.")

O time mandante faria 1.3989983305509182 gols
O RMSE do modelo baseline foi de 1.0373648770321195 gols.


### Regressão Linear

In [693]:
from sklearn.linear_model import LinearRegression

# Cria um modelo linear
modelo_linear = LinearRegression()

# Treina esse modelo 
modelo_linear.fit(X_treino, y_treino)

# Realiza uma previsão usando o modelo treinado
previsao_linear = modelo_linear.predict(X_teste)
print(f"O time mandante faria {previsao_linear[0]} gols") # Foi escolhida apenas uma linha para melhorar a visualização

# Verificando o erro do modelo usando o RMSE (root mean squared error)

y_verdadeiro = y_teste
y_previsao_linear = modelo_linear.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao_linear, squared=False)

print(f"O RMSE do modelo linear foi de {RMSE} gols.")

O time mandante faria [1.89004739] gols
O RMSE do modelo linear foi de 1.0238865067603353 gols.


### Regressão Linear Normalizada

In [694]:
from sklearn.preprocessing import MinMaxScaler

# Usa um função normalizadora sobre os dados

normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

# Cria um modelo linear
modelo_linear_norm = LinearRegression()

# Treina esse modelo
modelo_linear_norm.fit(
    normalizador_x.transform(X_treino),
    normalizador_y.transform(y_treino),
)

# Realiza uma previsão usando o modelo linear normalizado treinado
previsao = modelo_linear_norm.predict(normalizador_x.transform(X_treino))
previsao = normalizador_y.inverse_transform(previsao)
print(previsao[0])

# Verificando o erro do modelo usando o RMSE (root mean squared error)

y_verdadeiro = y_teste
y_previsao = modelo_linear_norm.predict(normalizador_x.transform(X_teste))
y_previsao = normalizador_y.inverse_transform(y_previsao)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo linear foi de {RMSE} gols.")

[1.25510974]
O RMSE do modelo linear foi de 1.0238865067545295 gols.
